In [1]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Conv1D, BatchNormalization, Activation, Add, Input, GlobalAveragePooling1D, Dense, MaxPooling1D
from tensorflow.keras.models import Model
import pickle
import shutil
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from matplotlib.colors import ListedColormap

In [2]:
def load_waveform_data(data_dir):
    """
    波形データを読み込む。pickleファイルが存在する場合はpickleファイルから、
    存在しない場合はExcelファイルから読み込み、pickleファイルを作成する。

    Args:
        data_dir (str): データディレクトリのパス

    Returns:
        tuple: 波形データとラベルのタプル
    """
    pickle_file = os.path.join(data_dir, 'waveform_data.pickle')
    if os.path.exists(pickle_file):
        with open(pickle_file, 'rb') as f:
            waveforms, labels = pickle.load(f)
    else:
        normal_waveforms = []
        abnormal_waveforms = []

        normal_path = os.path.join(data_dir, 'dataset_normal')
        for filename in os.listdir(normal_path):
            if filename.endswith('.xlsx'):
                filepath = os.path.join(normal_path, filename)
                df = pd.read_excel(filepath, header=None)
                waveform = df.values
                normal_waveforms.append(waveform)

        abnormal_path = os.path.join(data_dir, 'dataset_anomaly')
        for filename in os.listdir(abnormal_path):
            if filename.endswith('.xlsx'):
                filepath = os.path.join(abnormal_path, filename)
                df = pd.read_excel(filepath, header=None)
                waveform = df.values
                abnormal_waveforms.append(waveform)

        waveforms = np.array(list(normal_waveforms) + abnormal_waveforms)
        labels = np.array([0] * len(normal_waveforms) + [1] * len(abnormal_waveforms))

        with open(pickle_file, 'wb') as f:
            pickle.dump((waveforms, labels), f)

    return waveforms, labels

data_dir = r"C:\Users\r-fujita\Desktop\ET\Phase3"
waveforms, labels = load_waveform_data(data_dir)

In [4]:
# インデックスの配列を作成
indices = np.arange(len(waveforms))

# データをトレーニングセットとテストセットに分割、インデックスも分割
X_train, X_test, y_train, y_test, indices_train, indices_test = train_test_split(
    waveforms, labels, indices, test_size=0.2, random_state=42
)

# ファイル名とデータを対応付ける
all_files = [os.path.basename(f) for f in os.listdir(os.path.join(data_dir, 'dataset_normal'))] + \
              [os.path.basename(f) for f in os.listdir(os.path.join(data_dir, 'dataset_anomaly'))]

# 分割されたインデックスを使用してファイル名を取得
train_files = [all_files[i] for i in indices_train]
test_files = [all_files[i] for i in indices_test]

train_data = {
    'files': train_files,
    'X': X_train,
    'y': y_train,
    'indices': indices_train  # インデックスを追加
}
test_data = {
    'files': test_files,
    'X': X_test,
    'y': y_test,
    'indices': indices_test  # インデックスを追加
}

# トレーニングデータとテストデータを保存
with open(os.path.join(data_dir, 'train_data.pickle'), 'wb') as f:
    pickle.dump(train_data, f)
with open(os.path.join(data_dir, 'test_data.pickle'), 'wb') as f:
    pickle.dump(test_data, f)

In [31]:
def resnet_block(input_tensor, filters, kernel_size, stride=1):
    """
    ResNetブロックを構築する。

    Args:
        input_tensor (Tensor): 入力テンソル
        filters (int): フィルターの数
        kernel_size (int): カーネルサイズ
        stride (int, optional): ストライド. Defaults to 1.

    Returns:
        Tensor: 出力テンソル
    """
    x = Conv1D(filters, kernel_size, strides=stride, padding='same')(input_tensor)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Conv1D(filters, kernel_size, strides=1, padding='same')(x)
    x = BatchNormalization()(x)

    if stride != 1 or input_tensor.shape[-1] != filters:
        shortcut = Conv1D(filters, kernel_size=1, strides=stride, padding='same')(input_tensor)
        shortcut = BatchNormalization()(shortcut)
    else:
        shortcut = input_tensor

    x = Add()([x, shortcut])
    x = Activation('relu')(x)
    return x

In [32]:
def create_resnet_model(input_shape):
    """
    ResNetベースの1次元CNNモデルを構築する。

    Args:
        input_shape (tuple): 入力形状

    Returns:
        Model: モデル
    """
    inputs = Input(shape=input_shape)
    x = Conv1D(64, kernel_size=7, strides=2, padding='same')(inputs)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = MaxPooling1D(pool_size=3, strides=2, padding='same')(x)

    x = resnet_block(x, filters=64, kernel_size=3)
    x = resnet_block(x, filters=128, kernel_size=3, stride=2)
    x = resnet_block(x, filters=256, kernel_size=3, stride=2)
    x = resnet_block(x, filters=512, kernel_size=3, stride=2)

    x = GlobalAveragePooling1D()(x)
    outputs = Dense(1, activation='sigmoid')(x)

    model = Model(inputs, outputs)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [33]:
# データ型の明示的な変換
X_train = X_train.astype(float)
y_train = y_train.astype(float)

# モデルの作成とトレーニング
input_shape = (200, 8)
model = create_resnet_model(input_shape)
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2)

Epoch 1/50
490/490 [==============================] - 21s 38ms/step - loss: 0.1205 - accuracy: 0.9688 - val_loss: 0.0978 - val_accuracy: 0.9706
Epoch 2/50
490/490 [==============================] - 17s 35ms/step - loss: 0.0644 - accuracy: 0.9814 - val_loss: 0.1277 - val_accuracy: 0.9694
Epoch 3/50
490/490 [==============================] - 17s 35ms/step - loss: 0.0483 - accuracy: 0.9865 - val_loss: 0.0601 - val_accuracy: 0.9831
Epoch 4/50
490/490 [==============================] - 18s 37ms/step - loss: 0.0383 - accuracy: 0.9893 - val_loss: 0.0392 - val_accuracy: 0.9888
Epoch 5/50
490/490 [==============================] - 18s 37ms/step - loss: 0.0330 - accuracy: 0.9902 - val_loss: 0.0293 - val_accuracy: 0.9916
Epoch 6/50
490/490 [==============================] - 18s 37ms/step - loss: 0.0288 - accuracy: 0.9913 - val_loss: 0.0600 - val_accuracy: 0.9834
Epoch 7/50
490/490 [==============================] - 18s 37ms/step - loss: 0.0250 - accuracy: 0.9923 - val_loss: 0.0266 - val_accuracy:

In [3]:
model = tf.keras.models.load_model(r"C:\Users\r-fujita\Desktop\ET\Phase3") 

In [6]:
# 訓練済みモデルを読み込む
#model = tf.keras.models.load_model('trained_model.h5')  # 訓練済みモデルのファイルパスを指定

def create_dataset(waveform_data, window_size, stride):
    """
    tf.data.Dataset を作成します。

    Args:
        waveform_data (tf.Tensor): 波形データ
        window_size (int): ウィンドウサイズ
        stride (int): ストライド

    Returns:
        tf.data.Dataset: データセット
    """
    dataset = tf.data.Dataset.from_tensor_slices(
        tf.signal.frame(waveform_data, window_size, stride, pad_end=True, pad_value=0.0)
    )
    return dataset.batch(1, drop_remainder=True).prefetch(tf.data.AUTOTUNE)  # バッチサイズを指定、prefetch を追加

def evaluate_waveform(waveform_data, labels, model, window_size, stride, threshold):
    """
    波形データに対してスライディングウィンドウ形式で精度を確認します。

    Args:
        waveform_data (np.ndarray): 波形データ
        labels (np.ndarray): ラベルデータ
        model: 訓練済みモデル
        window_size (int): ウィンドウサイズ
        stride (int): ストライド
        threshold (float): 閾値
    """

    dataset = create_dataset(waveform_data, window_size, stride)

    anomaly_scores = []
    for batch_windows in dataset:
        predictions = model.predict(batch_windows)
        anomaly_scores.extend(predictions.flatten())

    # 予測結果をクラスラベルに変換
    predicted_labels = np.where(np.array(anomaly_scores) > threshold, 1, 0)

    # 各ウィンドウの中央位置を計算
    window_centers = np.arange(window_size // 2, len(waveform_data) - window_size // 2 + 1, stride)

    # 正解ラベルをウィンドウに合わせてリサイズ
    resized_labels = []
    for center in window_centers:
        resized_labels.append(labels[center])  # 各ウィンドウの中央に対応するラベルを取得
    resized_labels = np.array(resized_labels)  # NumPy配列に変換

    # 正解率を計算
    correct_predictions = np.sum(predicted_labels == resized_labels)  # 正解数を計算
    accuracy = correct_predictions / len(resized_labels)  # 正解率を計算

    # 結果を表示
    print("Accuracy:", accuracy)  # 正解率を表示

def evaluate_test_data(data_dir, model, window_size, stride, threshold):
    """
    テストデータに対してスライディングウィンドウ形式で精度を確認します。

    Args:
        data_dir (str): データディレクトリのパス
        model: 訓練済みモデル
        window_size (int): ウィンドウサイズ
        stride (int): ストライド
        threshold (float): 閾値
    """

    with open(os.path.join(data_dir, 'test_data.pickle'), 'rb') as f:
        test_data = pickle.load(f)

    for waveform_data, labels in zip(test_data['X'], test_data['y']):
        evaluate_waveform(waveform_data, labels, model, window_size, stride, threshold)

if __name__ == "__main__":
    data_dir = r"C:\Users\r-fujita\Desktop\ET\Phase3"  # 必要に応じてパスを修正してください
    window_size = 200
    stride = 50
    threshold = 0.75
    evaluate_test_data(data_dir, model, window_size, stride, threshold)

ValueError: in user code:

    File "C:\Program Files\Python310\lib\site-packages\keras\engine\training.py", line 2041, in predict_function  *
        return step_function(self, iterator)
    File "C:\Program Files\Python310\lib\site-packages\keras\engine\training.py", line 2027, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Program Files\Python310\lib\site-packages\keras\engine\training.py", line 2015, in run_step  **
        outputs = model.predict_step(data)
    File "C:\Program Files\Python310\lib\site-packages\keras\engine\training.py", line 1983, in predict_step
        return self(x, training=False)
    File "C:\Program Files\Python310\lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "C:\Program Files\Python310\lib\site-packages\keras\engine\input_spec.py", line 295, in assert_input_compatibility
        raise ValueError(

    Val

In [10]:
len(test_data)

8879